# Connectome Analyses

We develop tools to analyze a circuit model's connectome.

In [1]:
import os
import numpy as np
import pandas as pd
from neuro_dmt.library.users.visood.sscx_dissemination.analyses.composition\
    import CompositionAnalysesSuite
from neuro_dmt.library.users.visood.sscx_dissemination.analyses.connectome\
    import ConnectomeAnalysesSuite
from neuro_dmt.models.bluebrain.circuit.atlas import\
    BlueBrainCircuitAtlas
from neuro_dmt.models.bluebrain.circuit.model import\
    BlueBrainCircuitModel,\
    CircuitProvenance
from neuro_dmt.models.bluebrain.circuit.adapter import\
    BlueBrainCircuitAdapter

('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'roi', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'roi', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'roi', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mtype', 'etype', 'synapse_class')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'roi', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'roi')


In [2]:
project =\
    os.path.join(
        "/gpfs/bbp.cscs.ch/project")
proj_sscx_diss=\
    os.path.join(
        project,
        "proj83")
data_sscx_diss=\
    os.path.join(
        proj_sscx_diss,
        "data")
atlases_sscx_diss=\
    os.path.join(
        data_sscx_diss,
        "atlas/S1/MEAN")
circuits_sscx_diss=\
    os.path.join(
        proj_sscx_diss,
        "circuits")
atlas_bio_m =\
    BlueBrainCircuitAtlas(
        path=os.path.join(
                atlases_sscx_diss, "P14-MEAN"))
path_bio_m =\
    os.path.join(
        circuits_sscx_diss,
        "Bio_M/20191206")
circuit_bio_m =\
    BlueBrainCircuitModel(
        path_circuit_data=path_bio_m,
            provenance=CircuitProvenance(
                label="SSCxRatDisseminationBioM",
                authors=["BBP Team"],
                release_date="20191212",
                uri=path_bio_m,
                animal="Wistar Rat",
                age="P14",
                brain_region="SSCx"))
adapter =\
    BlueBrainCircuitAdapter()
suite_connectome_analyses =\
    ConnectomeAnalysesSuite(
        sample_size=20)

## Mock circuit

We will use a mock-circuit to test-develop connectome analyses.

In [3]:
from neuro_dmt.models.bluebrain.circuit.mock.circuit import\
    MockCircuit
from neuro_dmt.models.bluebrain.circuit.mock.test.mock_circuit_light import\
    circuit_composition,\
    circuit_connectivity

bluepy_mock_circuit =\
    MockCircuit.build(
        circuit_composition,
        circuit_connectivity)
mock_circuit_model =\
    BlueBrainCircuitModel(
        bluepy_mock_circuit,
        label="BlueBrainCircuitModelMockLight")

82304it [02:03, 664.16it/s]
100%|██████████| 82304/82304 [01:34<00:00, 871.93it/s]


# Adapter Functionality

In this section we develop our adapter's functionality

In [4]:
from neuro_dmt.models.bluebrain.circuit.adapter\
    import BlueBrainCircuitAdapter
adapter = BlueBrainCircuitAdapter()
cell_types_m = adapter.get_cell_types(mock_circuit_model, ("mtype",))
print(cell_types_m.head())
cell_types_me = adapter.get_cell_types(mock_circuit_model, ("mtype", "etype",) )
print(cell_types_me.head())

     mtype
0   L1_DAC
1  L1_SLAC
2   L23_MC
3  L23_ChC
4  L23_SBC
  etype    mtype
0  bNAC   L1_DAC
1   NAC   L1_DAC
2  bNAC  L1_SLAC
3   NAC  L1_SLAC
4  bNAC   L23_MC


In [5]:
pathways = adapter.get_pathways(mock_circuit_model, ("mtype", "etype"))
print(pathways.shape)
assert pathways.shape[0] == cell_types_me.shape[0] ** 2
pathways.head()

(1444, 4)


pre_synaptic         post_synaptic         
         etype   mtype         etype    mtype
0         bNAC  L1_DAC          bNAC   L1_DAC
1         bNAC  L1_DAC           NAC   L1_DAC
2         bNAC  L1_DAC          bNAC  L1_SLAC
3         bNAC  L1_DAC           NAC  L1_SLAC
4         bNAC  L1_DAC          bNAC   L23_MC

In [6]:
pathways.iloc[1]

pre_synaptic   etype      bNAC
               mtype    L1_DAC
post_synaptic  etype       NAC
               mtype    L1_DAC
Name: 1, dtype: object

In [7]:
adapter.get_cells(mock_circuit_model,
                  **pathways.iloc[1].post_synaptic)\
        .etype\
        .unique()

array(['NAC'], dtype=object)

# Number of Afferent Connections

We develop an analysis of the number of afferent connections

In [8]:
from neuro_dmt.analysis.reporting import CheetahReporter
reporter_analyses =\
    CheetahReporter(
        path_output_folder=os.path.join(
            suite_connectome_analyses.path_reports,
            "analyses"))

We can also run our analyses over a chosen set of cell-types.

In [9]:
post_synaptic_cell =\
    suite_connectome_analyses\
        .random_cell(
            mock_circuit_model,
            adapter,
            {"mtype": "L23_MC"})
post_synaptic_cell

etype                    NAC
layer                      2
morph_class      not-defined
mtype                 L23_MC
nucleus          not-defined
region                  S1Sh
synapse_class            INH
x                    196.675
y                    212.023
z                    2.38328
gid                    62165
Name: 62165, dtype: object

In [10]:
cells = adapter.get_cells(mock_circuit_model)
post_synaptic_cell = cells.iloc[0]
m =\
    suite_connectome_analyses\
        .number_connections_afferent(
            mock_circuit_model, adapter,
            cells.iloc[0],
            cell_properties_groupby=["mtype"])
m.head()

100%|██████████| 82304/82304 [00:06<00:00, 13667.90it/s]


mtype    soma_distance
L1_DAC   50.0             1.0
L1_SLAC  150.0            2.0
L23_ChC  50.0             1.0
         250.0            1.0
         350.0            1.0
Name: number_connections_afferent, dtype: float64

In [41]:
from neuro_dmt.library.users.visood.sscx_dissemination.analyses.tools\
    import PathwayMeasurement

pathway_measurement =\
    PathwayMeasurement(
        value=lambda connections: connections.strength.to_numpy(dtype=np.float64),#suite_connectome_analyses.number_connections_afferent,
        sample_size=20)

ms =[
    m for m in pathway_measurement.sample(
        mock_circuit_model,
        adapter,
        post_synaptic_cell={"mtype": "L23_MC"})] 
pd.concat(ms, axis=1).head()

41994  412    21025  20905  62583  \
(pre_synaptic_cell, mtype) soma_distance                                      
L1_DAC                     50.0             NaN    NaN   10.0    NaN    NaN   
                           150.0            NaN    NaN   12.0    NaN    NaN   
                           250.0            NaN    NaN    4.0    NaN    8.0   
                           350.0            NaN    NaN    NaN    NaN    NaN   
                           450.0            NaN    NaN    NaN    NaN    NaN   

                                          20886  329    930    401    308    \
(pre_synaptic_cell, mtype) soma_distance                                      
L1_DAC                     50.0             6.0    NaN    NaN    NaN    NaN   
                           150.0           11.0    3.0    NaN   11.0    5.0   
                           250.0            NaN    NaN    NaN    NaN    8.0   
                           350.0            NaN    NaN    NaN    NaN    NaN   
                           450.0            NaN    NaN    2.0    NaN    NaN   

                                          423    290    62136  811    62561  \
(pre_synaptic_cell, mtype) soma_distance                                      
L1_DAC                     50.0             NaN    7.0    NaN    NaN    NaN   
                           150.0           12.0    4.0    NaN    NaN    NaN   
                           250.0            9.0    NaN    5.0    NaN    NaN   
                           350.0            NaN    NaN    NaN    NaN    NaN   
                           450.0            NaN    NaN    NaN    4.0    NaN   

                                          62012  42000  21389  21508  41960  
(pre_synaptic_cell, mtype) soma_distance                                     
L1_DAC                     50.0             NaN    NaN    NaN    NaN    NaN  
                           150.0            8.0    NaN    NaN    NaN    NaN  
                           250.0            NaN   12.0    NaN    NaN    NaN  
                           350.0            NaN    NaN    NaN    9.0    NaN  
                           450.0            NaN    NaN    9.0    NaN    NaN

In [43]:
pathway_measurement.sample_one(mock_circuit_model,
                               adapter,
                               post_synaptic_cell={"mtype": "L23_MC"})\
                   .max()

89.0

In [28]:
pathway_measurement.collect(mock_circuit_model,
                            adapter,
                            post_synaptic_cell={"mtype": "L23_MC"})\
                   .head()

819    21423  21006  317    42026  \
(pre_synaptic_cell, mtype) soma_distance                                      
L1_DAC                     50.0             NaN    NaN    1.0    NaN    NaN   
                           150.0            NaN    NaN    NaN    NaN    NaN   
                           250.0            NaN    2.0    NaN    1.0    4.0   
                           350.0            NaN    NaN    NaN    NaN    NaN   
                           450.0            2.0    NaN    NaN    NaN    NaN   

                                          62113  62092  62641  21457  328    \
(pre_synaptic_cell, mtype) soma_distance                                      
L1_DAC                     50.0             NaN    NaN    NaN    NaN    NaN   
                           150.0            2.0    NaN    1.0    NaN    NaN   
                           250.0            NaN    NaN    NaN    NaN    1.0   
                           350.0            NaN    NaN    1.0    NaN    NaN   
                           450.0            NaN    NaN    NaN    NaN    NaN   

                                          409    20889  900    62125  839    \
(pre_synaptic_cell, mtype) soma_distance                                      
L1_DAC                     50.0             NaN    NaN    NaN    1.0    NaN   
                           150.0            NaN    NaN    NaN    NaN    NaN   
                           250.0            1.0    NaN    2.0    NaN    NaN   
                           350.0            NaN    NaN    NaN    NaN    1.0   
                           450.0            NaN    NaN    NaN    NaN    NaN   

                                          41529  62648  20950  821    41601  
(pre_synaptic_cell, mtype) soma_distance                                     
L1_DAC                     50.0             NaN    NaN    NaN    NaN    NaN  
                           150.0            4.0    NaN    NaN    NaN    NaN  
                           250.0            1.0    NaN    NaN    1.0    NaN  
                           350.0            NaN    1.0    NaN    NaN    NaN  
                           450.0            NaN    1.0    NaN    NaN    NaN

In [29]:
pathway_measurement.summary(mock_circuit_model,
                            adapter,
                            post_synaptic_cell={"mtype": "L23_MC"})\
                   .head()

count   sum      mean       mad       std       var  min  \
L1_DAC 50.0     1.0   2.0  2.000000  0.000000       NaN       NaN  2.0   
       150.0    7.0   9.0  1.285714  0.408163  0.487950  0.238095  1.0   
       250.0   10.0  14.0  1.400000  0.480000  0.516398  0.266667  1.0   
       450.0    3.0   5.0  1.666667  0.444444  0.577350  0.333333  1.0   
       550.0    2.0   2.0  1.000000  0.000000  0.000000  0.000000  1.0   

              median  max  
L1_DAC 50.0      2.0  2.0  
       150.0     1.0  2.0  
       250.0     1.0  2.0  
       450.0     2.0  2.0  
       550.0     1.0  1.0

In [16]:
analysis_naff =\
    suite_connectome_analyses\
        .analysis_number_connections_afferent
analysis_naff(
    mock_circuit_model,
    adapter)

Please provide Field 'value':
                        
		        Measurement method for a single set of parameter-values.
		        


ValueError: 
                    Cannot create 'PathwayMeasurement' instance without required Field 'value'.
                    Please provide a value as a keyword argument in your 
                    instance initialization.
                    Missing Field 'value':
                    	
		        Measurement method for a single set of parameter-values.
		        
                    

O

## SSCx variation

In [ ]:
report_connectome_analyses_bio_m =\
    suite_connectome_analyses\
        .analysis_number_afferent_connections(
            circuit_bio_m,
            adapter)


Because 60 post-synaptic mtypes can take too long, we can set the number of 
mtypes to process in one run:

In [ ]:
def get_layer_mtypes(circuit, query_layer):
    mtypes_all =\
        pd.Series(adapter.get_mtypes(circuit), name="mtype")
    return\
        list(mtypes_all[
                [query_layer in mtype for mtype in mtypes_all]
        ].values)
def analyze_number_afferent_connections(
        circuit,
        adapter,
        selected_mtypes,
        reporter,
        **kwargs):
    """..."""
    suite_connectome_analyses =\
        ConnectomeAnalysesSuite(
            sample_size=20,
            cell_mtypes=selected_mtypes)
    report=\
        suite_connectome_analyses\
            .analysis_number_afferent_connections(
                circuit, adapter, **kwargs)
    return reporter.post(report)

In [ ]:
analyze_number_afferent_connections(
    mock_circuit_model,
    adapter,
    ["L5_TPC:A", "L5_TPC:B",  "L5_UPC"],
    reporter_analyses,
    sample_size=200)

In [ ]:
suite_connectome_analyses =\
    ConnectomeAnalysesSuite(
        sample_size=20,
        cell_mtypes=["L23_MC"])
measurement_example =\
    suite_connectome_analyses\
        .analysis_number_afferent_connections\
        .get_measurement(
            mock_circuit_model,
            adapter)

In [ ]:
measurement_example["data"]